# 🎯 Data Quality & Joins - Interview Scenarios

## **Module Overview**
**Master data quality handling, complex joins, and advanced transformations for robust PySpark applications.**

**🎯 Difficulty**: 🟡 Intermediate
**⏱️ Estimated Time**: 2-3 hours
**🎓 Learning Focus**: Data quality and complex join operations

---


## 🎯 What You Will Learn

By completing this module, you will master:
- ✅ **Complex join operations** (self-joins, multi-table joins)
- ✅ **Data quality handling** (nulls, duplicates, malformed data)
- ✅ **Advanced transformations** (explode, flatten, pivot)
- ✅ **JSON processing** and nested structures
- ✅ **Error handling** and data validation

---


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Data Quality Joins").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


# 🚀 Scenario 1: Remove Duplicate Records

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement remove duplicate records in PySpark"**

### **💼 Business Context:**
Handle duplicate data and ensure data uniqueness - common requirement in data engineering roles.

---


In [ ]:
# Scenario 1: Remove duplicate employees by name

employee_data = [
    (1, "Jhon", "Testing", 5000),
    (2, "Tim", "Development", 6000),
    (3, "Jhon", "Development", 5000),
    (4, "Sky", "Production", 8000)
]

df_employees = spark.createDataFrame(employee_data, ["id", "name", "dept", "salary"])
df_employees.show()

# Remove duplicates based on name, keeping first occurrence
df_unique = df_employees.dropDuplicates(["name"]).orderBy("id")
df_unique.show()


# 🚀 Scenario 2: Complex Join Operations

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement complex join operations in PySpark"**

### **💼 Business Context:**
Master different types of joins and their use cases - common requirement in data engineering roles.

---


In [ ]:
# Scenario 2: Outer join employee details

emp_personal = [
    (1, "Tim", 24, "Kerala", "India"),
    (2, "Asman", 26, "Kerala", "India")
]

emp_address = [
    (1, "Tim", 24, "Comcity"),
    (2, "Asman", 26, "bimcity")
]

df_personal = spark.createDataFrame(emp_personal, ["emp_id", "name", "age", "state", "country"])
df_address = spark.createDataFrame(emp_address, ["emp_id", "name", "age", "address"])

df_personal.show()
df_address.show()

# Outer join on multiple columns
df_joined = df_personal.join(df_address, ["emp_id", "name", "age"], "outer")
df_joined.show()


# 🚀 Scenario 3: Word Processing with UDFs

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement word processing with udfs in PySpark"**

### **💼 Business Context:**
Implement custom functions for text processing - common requirement in data engineering roles.

---


In [ ]:
# Scenario 3: Reverse words in sentence using UDF

sentence_data = [("The Social Dilemma",)]

df_input = spark.createDataFrame(sentence_data, ["word"])
df_input.show()

def reverse_sentence(sentence):
    """Reverse each word in the sentence"""
    return " ".join([word[::-1] for word in sentence.split(" ")])

# Register UDF
reverse_udf = udf(reverse_sentence, StringType())

# Apply UDF
df_output = df_input.withColumn("reversed_word", reverse_udf("word")).drop("word")
df_output.show()


# 🚀 Scenario 4: JSON Data Flattening

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement json data flattening in PySpark"**

### **💼 Business Context:**
Transform nested JSON structures to flat tables - common requirement in data engineering roles.

---


In [ ]:
# Scenario 4: Flatten nested JSON data

import json

json_data = {
    "code": 1234,
    "commentCount": 5,
    "id": 1,
    "name": "John Doe",
    "likeDislike": {
        "dislikes": 10,
        "likes": 20,
        "userAction": 1
    },
    "multiMedia": [
        {
            "id": 1001,
            "name": "Media name",
            "url": "https://example.com/media1"
        },
        {
            "id": 1002,
            "name": "Another media name",
            "url": "https://example.com/media2"
        }
    ]
}

# Create DataFrame from JSON
df_json = spark.read.json(spark.sparkContext.parallelize([json.dumps(json_data)]))
df_json.printSchema()

# Flatten nested structures
df_flattened = df_json.withColumn("multiMedia", explode(col("multiMedia")))\
    .withColumn("dislikes", col("likeDislike.dislikes"))\
    .withColumn("likes", col("likeDislike.likes"))\
    .withColumn("userAction", col("likeDislike.userAction"))\
    .withColumn("media_id", col("multiMedia.id"))\
    .withColumn("media_name", col("multiMedia.name"))\
    .withColumn("media_url", col("multiMedia.url"))\
    .drop("likeDislike", "multiMedia")

df_flattened.printSchema()
df_flattened.show()


# 🚀 Scenario 5: Advanced JSON Restructuring

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement advanced json restructuring in PySpark"**

### **💼 Business Context:**
Create complex nested data structures - common requirement in data engineering roles.

---


In [ ]:
# Scenario 5: Restructure complex JSON data

json_list = [
    {
        "id": 1, "name": "John", "likes": 10, "dislikes": 2,
        "createAt": "2023-05-30T10:30:00"
    },
    {
        "id": 2, "name": "Jane", "likes": 15, "dislikes": 1,
        "createAt": "2023-05-31T11:00:00"
    }
]

# Create DataFrame from JSON list
df_json_list = spark.read.json(spark.sparkContext.parallelize([json.dumps(item) for item in json_list]))
df_json_list.printSchema()
df_json_list.show()

# Restructure with nested structures
df_restructured = df_json_list.select(
    col("id"),
    col("name"),
    struct(col("dislikes"), col("likes"), col("userAction")).alias("likeDislike"),
    array(struct(col("createAt"), col("name"), col("id"))).alias("multiMedia")
)

df_restructured.printSchema()


# 🚀 Scenario 6: Round Trip Calculations

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement round trip calculations in PySpark"**

### **💼 Business Context:**
Complex join operations for route analysis - common requirement in data engineering roles.

---


In [ ]:
# Scenario 6: Calculate round trip distances

trip_data = [
    ("SEA", "SF", 300),
    ("CHI", "SEA", 2000),
    ("SF", "SEA", 300),
    ("SEA", "CHI", 2000),
    ("SEA", "LND", 500),
    ("LND", "SEA", 500),
    ("LND", "CHI", 1000),
    ("CHI", "NDL", 180)
]

df_trips = spark.createDataFrame(trip_data, ["from", "to", "dist"])
df_trips.show()

# Method 1: SQL Approach
df_trips.createOrReplaceTempView("trip")
spark.sql("""
    SELECT r1."from", r1."to", (r1.dist + r2.dist) AS roundtrip_dist
    FROM trip r1
    JOIN trip r2 ON r1."from" = r2."to" AND r1."to" = r2."from"
    WHERE r1."from" < r1."to"
""").show()

# Method 2: DSL Approach
df_roundtrip = df_trips.alias("r1").join(
    df_trips.alias("r2"),
    (col("r1.from") == col("r2.to")) & (col("r1.to") == col("r2.from")),
    "inner"
).where(col("r1.from") < col("r1.to"))\
 .select(col("r1.from"), col("r1.to"), (col("r1.dist") + col("r2.dist")).alias("roundtrip_dist"))

df_roundtrip.show()


# 🚀 Scenario 7: Running Totals with Window Functions

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement running totals with window functions in PySpark"**

### **💼 Business Context:**
Calculate cumulative sums over partitions - common requirement in data engineering roles.

---


In [ ]:
# Scenario 7: Calculate running totals

order_data = [
    (1, "26-May", 100),
    (1, "27-May", 200),
    (1, "28-May", 300),
    (2, "29-May", 400),
    (3, "30-May", 500),
    (3, "31-May", 600)
]

df_orders = spark.createDataFrame(order_data, ["pid", "date", "price"])
df_orders.show()

# Method 1: SQL Approach
df_orders.createOrReplaceTempView("ordertab")
spark.sql("""
    SELECT pid, date, price,
           SUM(price) OVER (PARTITION BY pid ORDER BY price) AS running_total
    FROM ordertab
""").show()

# Method 2: DSL Approach
window_spec = Window.partitionBy("pid").orderBy("price")
df_with_running_total = df_orders.withColumn("running_total", sum("price").over(window_spec))
df_with_running_total.show()


# 🚀 Scenario 8: Customer Analysis

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement customer analysis in PySpark"**

### **💼 Business Context:**
Multi-table joins for business intelligence - common requirement in data engineering roles.

---


In [ ]:
# Scenario 8: Find customers who bought all products

purchase_data = [
    (1, 5), (2, 6), (3, 5), (3, 6), (1, 6)
]

product_data = [(5,), (6,)]

df_purchases = spark.createDataFrame(purchase_data, ["customer_id", "product_key"])
df_products = spark.createDataFrame(product_data, ["product_key"])

df_purchases.show()
df_products.show()

# Find customers who bought products (excluding customer 2)
df_customer_products = df_purchases.join(df_products, "product_key", "inner")\
    .select("customer_id", "product_key")\
    .distinct()\
    .where(col("customer_id") != 2)

df_customer_products.show()


# 🚀 Scenario 9: Page Visit Analysis

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement page visit analysis in PySpark"**

### **💼 Business Context:**
Array operations and collections - common requirement in data engineering roles.

---


In [ ]:
# Scenario 9: Collect user page visits

page_visit_data = [
    (1, "home"),
    (1, "products"),
    (1, "checkout"),
    (1, "confirmation"),
    (2, "home"),
    (2, "products"),
    (2, "cart"),
    (2, "checkout"),
    (2, "confirmation"),
    (2, "home"),
    (2, "products")
]

df_page_visits = spark.createDataFrame(page_visit_data, ["userid", "page"])
df_page_visits.show()

# Method 1: SQL Approach
df_page_visits.createOrReplaceTempView("pagetab")
spark.sql("SELECT userid, COLLECT_LIST(page) AS pages FROM pagetab GROUP BY userid").show()

# Method 2: DSL Approac
df_user_pages = df_page_visits.groupBy("userid")\
    .agg(collect_list("page").alias("pages"))

df_user_pages.show(truncate=False)


# 🚀 Scenario 10: CSV Error Handling

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement csv error handling in PySpark"**

### **💼 Business Context:**
Process malformed CSV data with error tolerance - common requirement in data engineering roles.

---


In [ ]:
# Scenario 10: Handle CSV with malformed records

csv_data = """emp_no,emp_name,dep
101,Murugan,HealthCare
Invalid Entry,Description: Bad Record Entry
102,Kannan,Finance
103,Mani,IT"""

# Create DataFrame from inline CSV with error handling
lines = csv_data.splitlines()
rdd = spark.sparkContext.parallelize(lines)
df_csv = spark.read.csv(rdd, header=True, mode="DROPMALFORMED")
df_csv.show()


# 🎉 Module Complete!

## **🏆 Congratulations!**

You have successfully completed the **Data Quality & Joins** interview scenarios module!

### **📊 What You Mastered:**
- **Complex join strategies** for multi-table operations
- **Data quality assurance** and error handling
- **JSON processing** and nested data structures
- **Advanced transformations** for data pipelines

### **🚀 Next Steps:**
Ready for complex business logic? Try the **Advanced Business Logic** module!

---
